### Layerwise Learning Rate Decay

Layerwise learning rate decay is a method used in training deep neural networks to adjust the learning rate for each layer individually. This technique enhances the convergence and performance of the model by allowing different layers to update at varying rates based on their relevance or sensitivity to the input data.

**How It Works:**

1. **Initial Learning Rate**: A global learning rate is set, which acts as the starting point for updating all parameters in the network during training.

2. **Decay Factor**: A decay factor is defined, typically a value between 0 and 1. This factor determines how much to reduce the learning rate for each layer.

3. **Layer-Specific Learning Rates**: Instead of applying a uniform learning rate to all layers, layer-specific learning rates are calculated. These rates are obtained by multiplying the initial learning rate by the decay factor raised to the power of the layer index.

4. **Application**: During training, each layer in the neural network utilizes its respective learning rate to update its parameters. Early layers, closer to the input, may have higher learning rates, while later layers, closer to the output, may have lower learning rates.

**Benefits:**

- **Preventing Overfitting**: Adjusting learning rates based on layer importance can prevent overfitting by controlling parameter updates effectively.
  
- **Speeding Up Convergence**: Tailoring learning rates to layer characteristics can accelerate convergence, leading to faster training times.

- **Improving Performance**: Layerwise learning rate decay can enhance the overall performance of the model by fine-tuning parameter updates.

Implementing layerwise learning rate decay involves defining initial learning rates, decay factors, and incorporating a learning rate scheduler into the training process.

In [1]:
# Importing the os module to access environment variables
import os

# Setting an environment variable to allow multiple instances of OpenMP libraries to be loaded
# This is a workaround for certain macOS environments where OpenMP might cause conflicts
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
# Importing necessary libraries
import time       # for time-related functions
import math       # for mathematical functions
import datetime   # for datetime operations
from collections import OrderedDict  # for ordered dictionaries
from tqdm.auto import tqdm   # for progress bars
from pathlib import Path     # for working with file paths
from thop import profile    # for calculating model FLOPs

# Importing PyTorch libraries
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.datasets.cifar import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import torch.nn.init as init   # for weight initialization

# Importing MLflow for experiment tracking
import mlflow

# Importing other utility libraries
import random     # for random number generation
import numpy as np   # for numerical operations
import psutil     # for system resource monitoring
import subprocess   # for executing shell commands
from torchsummary import summary   # for model summary
import matplotlib.pyplot as plt   # for plotting
import seaborn as sns   # for advanced plotting
from sklearn.metrics import confusion_matrix, classification_report   # for evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score   # for evaluation metrics

# Importing Captum for model interpretability
from captum.attr import FeatureAblation
from captum.attr import Occlusion
from captum.attr import LayerConductance
from captum.attr import IntegratedGradients
from captum.attr import Saliency
from captum.attr import DeepLift
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

# Suppressing warnings
import warnings 
warnings.filterwarnings("ignore")

In [3]:
# torch.cuda.memory._record_memory_history()

In [4]:
def increment_path(path, exist_ok=False, sep='', mkdir=True):
    # Convert the path to a Path object
    path = Path(path)
    
    # If the path already exists and `exist_ok` is False, handle incrementing the path
    if path.exists() and not exist_ok:
        # Check if the path is a file or a directory
        path, suffix = (path.with_suffix(''), path.suffix) if path.is_file() else (path, '')
        # Loop over numbers starting from 2 to 9999 to find a unique name
        for n in range(2, 9999):
            # Construct the new path with the incremented number
            p = f'{path}{sep}{n}{suffix}'
            # Check if the constructed path exists, if not, break the loop
            if not os.path.exists(p):
                break
        # Update the path to the incremented path
        path = Path(p)
    
    # If `mkdir` is True, create the directory structure specified by the path
    if mkdir:
        path.mkdir(parents=True, exist_ok=True)
    
    # Return the incremented path
    return path

In [5]:
# Define the experiment name
EXPERIMENT_NAME = "exp"

# Get the current working directory and resolve its absolute path
FILE = Path(os.getcwd()).resolve()

# Create a save directory path by incrementing the path
SAVE_DIR = increment_path(FILE / "artifacts" / f"{EXPERIMENT_NAME}", exist_ok=False)

# Create a 'checkpoints' directory inside the save directory
(SAVE_DIR / 'checkpoints').mkdir(parents=True, exist_ok=True)

In [6]:
# Define the URI for local tracking using SQLite
local_tracking_uri = f"sqlite:///mlflow/mlflow.db"

# Set the tracking URI to the local URI
mlflow.set_tracking_uri(local_tracking_uri)

# Set the experiment name to the stem (last part) of the save directory path
mlflow.set_experiment(SAVE_DIR.stem)

2024/03/17 04:19:06 INFO mlflow.tracking.fluent: Experiment with name 'exp40' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Hp/Desktop/mldev/2024/Git/mldevworld/Deep-Learning-Essentials/mlruns/37', creation_time=1710629346304, experiment_id='37', last_update_time=1710629346304, lifecycle_stage='active', name='exp40', tags={}>

In [7]:
def set_experiment_seed(seed=42):
    """
    Set random seeds and CUDA-related flags for experiment reproducibility.
    """
    # Set random seeds for Python, NumPy, and random module
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # If GPU is available, set random seed for CUDA operations
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # Set random seed for CUDA operations
        torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior for CuDNN
        torch.backends.cudnn.benchmark = False  # Disable CuDNN benchmark mode for reproducibility

    # Optionally log seed information
    print(f"Random seed set to {seed}")

# Call the function to set the experiment seed
set_experiment_seed()

Random seed set to 42


In [ ]:
# Download CIFAR10 dataset for training with transformations
data_train = CIFAR10("./data/cifar", download=True, train=True, transform=transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))

# Download CIFAR10 dataset for testing with transformations
data_test = CIFAR10("./data/cifar", download=True, train=False, transform=transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))

# Define test time transformations
tta_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
])

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define hyperparameters
N_CLASSES = 10  # Number of classes in the dataset
EPOCHS = 10     # Number of training epochs
BATCH_SIZE = 64  # Batch size for training
DROPOUT_PROB = 0.1  # Probability of dropout
USE_BATCHNORM = True  # Whether to use batch normalization
LABEL_SMOOTHING = 0.1  # Label smoothing factor
BASE_LR = 0.003 # Learning rate for base model
HEAD_LR = 0.03 # Learning rate for model head
LR_DECAY_FACTOR = 0.5
WARMUP_PROPORTION = 0.05  # Proportion of warmup steps
WEIGHT_DECAY = 1e-6  # Weight decay factor
ACCUMULATION_STEPS = 2  # Number of gradient accumulation steps
N_TTA = 4  # Number of test-time augmentations

In [ ]:
import matplotlib.pyplot as plt

# Set the figure size
plt.figure(figsize=(10, 10))

# Iterate over the first 25 samples in the training dataset
for i in range(25):
    # Create subplots in a 5x5 grid
    plt.subplot(5, 5, i + 1)
    
    # Remove ticks and gridlines
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    
    # Display the image (permute is used to change the order of dimensions)
    plt.imshow(data_train[i][0].permute(1, 2, 0).numpy(), cmap=plt.cm.binary)
    
    # Add the label as xlabel
    plt.xlabel(data_train[i][1])

# Show the plot
plt.show()

In [ ]:
def calculate_num_workers():
    # Calculate the number of CPU cores
    num_cpu_cores = os.cpu_count()

    # Set a safe maximum multiplier value (e.g., 0.5) to avoid using all available resources
    max_multiplier = 0.5

    # Calculate the number of workers based on the available resources
    multiplier = min(sum(psutil.cpu_percent(interval=1, percpu=True)) / 100.0, max_multiplier)
    
    # Ensure that num_workers is at least 1
    num_workers = max(1, int(num_cpu_cores * multiplier))

    return num_workers

In [ ]:
# Define train dataloader
train_dataloader = DataLoader(
    data_train,   # Training dataset
    batch_size=BATCH_SIZE,   # Batch size for training
    sampler=RandomSampler(data_train),   # Random sampler for shuffling data
    pin_memory=True,   # Pin memory for faster data transfer to GPU
    num_workers=calculate_num_workers()   # Number of workers for data loading
)

# Define test dataloader
test_dataloader = DataLoader(
    data_test,   # Test dataset
    batch_size=BATCH_SIZE,   # Batch size for testing
    sampler=SequentialSampler(data_test),   # Sequential sampler for sequential testing
    pin_memory=True,   # Pin memory for faster data transfer to GPU
    num_workers=calculate_num_workers()   # Number of workers for data loading
)

In [ ]:
# Assign device to CUDA (GPU) if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Instantiate the ResNet-18 model with pretrained weights
model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the model
# for param in model.parameters():
#     param.requires_grad = False

# Get the number of input features for the fully connected layer
num_features = model.fc.in_features

# Replace the fully connected layer with a new linear layer for 10 classes
model.fc = nn.Linear(num_features, 10)

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)


# Display a summary of the model architecture
summary(model, input_size=(3, 224, 224), batch_size=BATCH_SIZE)

In [ ]:
class LabelSmoothedCrossEntropy(nn.Module):
    def __init__(self, num_classes, smoothing):
        super(LabelSmoothedCrossEntropy, self).__init__()
        self.num_classes = num_classes  # Number of classes
        self.smoothing = smoothing  # Smoothing factor
        self.confidence = 1.0 - smoothing  # Confidence in true labels

    def forward(self, input_logits, target):
        # Compute log probabilities using log softmax
        log_probs = F.log_softmax(input_logits, dim=-1)
        
        # Create a tensor of zeros with the same shape as log_probs
        true_dist = torch.zeros_like(log_probs)
        
        # Fill true_dist with the smoothing factor divided by (num_classes - 1)
        true_dist.fill_(self.smoothing / (self.num_classes - 1))
        
        # Scatter true labels with confidence in true_dist
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        
        # Compute the loss as the negative sum of true_dist times log_probs
        # Divide by the batch size to get the average loss per sample
        loss = -torch.sum(true_dist * log_probs) / input_logits.size(0)
        
        return loss

In [ ]:
# Initialize loss
# criterion = nn.CrossEntropyLoss()

# Using LabelSmoothedCrossEntropy instead of nn.CrossEntropyLoss
criterion = LabelSmoothedCrossEntropy(num_classes=N_CLASSES, smoothing=LABEL_SMOOTHING)

#### Adam w/ Weight Decay

In [ ]:
def get_optimizer_grouped_parameters():
    """
    Generate grouped parameters for the optimizer with layerwise learning rate decay.

    Returns:
        list: A list of dictionaries containing grouped parameters for the optimizer.
    """
    no_decay = ['bias', 'BatchNorm2d.weight', 'BatchNorm2d.bias']
    
    # Initialize list to store optimizer grouped parameters
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if "fc" in n],  # Parameters of the final fully connected layers
            "weight_decay": 0.0,  # No weight decay for these parameters
            "lr": HEAD_LR,  # Learning rate for the head
        },
    ]
    
    # Get ResNet layers in reverse order
    layers = [model.layer1, model.layer2, model.layer3, model.layer4]
    layers.reverse()
    
    lr = BASE_LR  # Initialize learning rate for layers
    for layer in layers:
        lr *= LR_DECAY_FACTOR  # Update learning rate with decay factor
        optimizer_grouped_parameters += [
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay) and not "fc" in n],  # Parameters with weight decay
                "weight_decay": WEIGHT_DECAY,
                "lr": lr,
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay) and not "fc" in n],  # Parameters without weight decay
                "weight_decay": 0.0,
                "lr": lr,
            },
        ]
    return optimizer_grouped_parameters

In [ ]:
# Initialize optimizer
# optimizer = optim.Adam(model.parameters(), lr=LR)

# Using AdamW optimizer instead of Adam
optimizer = optim.AdamW(get_optimizer_grouped_parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)

#### Cosine Annealing With Warmup

In [ ]:
# Define a function to calculate the learning rate lambda
def get_lr_lambda(initial_lr, warmup_steps, total_steps):
    def lr_lambda(current_step):
        # Linear warmup phase
        if current_step < warmup_steps:
            return initial_lr + (1.0 - initial_lr) * float(current_step) / float(max(1, warmup_steps))
        # Cosine annealing phase
        else:
            return max(0.0, 0.5 * (1.0 + math.cos(math.pi * (current_step - warmup_steps) / float(total_steps - warmup_steps))))
    return lr_lambda

In [ ]:
# Calculate the length of the dataloader
len_dataloader = len(train_dataloader)

# Calculate the number of update steps per epoch
num_update_steps_per_epoch = max(len_dataloader // ACCUMULATION_STEPS, 1)

# Calculate the total number of examples in the dataset
num_examples = len(train_dataloader.dataset)

# Calculate the maximum number of steps (epochs * update steps per epoch)
max_steps = math.ceil(EPOCHS * num_update_steps_per_epoch)

# Calculate the number of warmup steps
num_warmup_steps = math.ceil(max_steps * WARMUP_PROPORTION)

# Get the learning rate lambda function
lr_lambda = get_lr_lambda(BASE_LR, num_warmup_steps, max_steps)

# Create a scheduler with LambdaLR using the learning rate lambda function
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

#### Adaptive Gradient Clipping

In [ ]:
def unitwise_norm(x, norm_type=2.0):
    # Calculate the norm of the tensor x
    # If x is 1D or lower, return the norm directly
    if x.ndim <= 1:
        return x.norm(norm_type)
    # If x is higher-dimensional, calculate the norm along all dimensions except the batch dimension
    else:
        return x.norm(norm_type, dim=tuple(range(1, x.ndim)), keepdim=True)


def adaptive_clip_grad(parameters, clip_factor=0.01, eps=1e-3, norm_type=2.0):
    # If parameters is a single tensor, convert it to a list
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    # Iterate over the parameters
    for p in parameters:
        # If the gradient of the parameter is None, skip to the next parameter
        if p.grad is None:
            continue
        # Detach the parameter and its gradient
        p_data = p.detach()
        g_data = p.grad.detach()
        # Calculate the maximum allowed gradient norm
        max_norm = unitwise_norm(p_data, norm_type=norm_type).clamp_(min=eps).mul_(clip_factor)
        # Calculate the norm of the gradient
        grad_norm = unitwise_norm(g_data, norm_type=norm_type)
        # Clip the gradient if its norm exceeds the maximum allowed norm
        clipped_grad = g_data * (max_norm / grad_norm.clamp(min=1e-6))
        # Replace the gradient with the clipped gradient where necessary
        new_grads = torch.where(grad_norm < max_norm, g_data, clipped_grad)
        # Update the gradient of the parameter
        p.grad.detach().copy_(new_grads)

#### Logging Model Parameters

In [ ]:
def calculate_flops(input_size):
    # Create a random input tensor with the specified input size
    input_tensor = torch.randn(*input_size).to(device)
    # Profile the model to calculate FLOPs and parameters
    flops, params = profile(model, inputs=(input_tensor,))
    return flops

def calculate_disk_size():
    # Calculate the size of the model on disk in bytes
    disk_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    # Convert bytes to megabytes
    disk_size_mb = disk_size_bytes / (1024 * 1024)
    return disk_size_mb

def get_model_metrics():
    # Calculate the total number of parameters in the model
    num_params = sum(p.numel() for p in model.parameters())
    # Calculate the number of trainable parameters in the model
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Set an example input size (adjust based on your actual input size)
    input_size = tuple(next(iter(train_dataloader))[0][0].unsqueeze(0).shape)

    # Calculate the FLOPs of the model
    flops = calculate_flops(input_size)
    # Calculate the disk size of the model
    disk_size = calculate_disk_size()

    # Return model metrics as a dictionary
    return {
        "num_params": num_params,
        "num_trainable_params": num_trainable_params,
        "flops": flops,
        "disk_size": disk_size
    }

#### Logging GPU Usage

In [ ]:
def get_gpu_usage():
    try:
        # Run the nvidia-smi command to get GPU utilization information
        result = subprocess.run(["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,nounits,noheader"], stdout=subprocess.PIPE, text=True)
        
        # Extract the GPU usage percentage from the command output
        gpu_usage = int(result.stdout.strip())
        
        # Return the GPU usage percentage
        return gpu_usage
    except Exception as e:
        # Print an error message if an exception occurs
        print(f"Error getting GPU usage: {e}")
        return None

#### Train

##### Gradient Penalty
Its primary purpose is to encourage smoothness in the learned representations of the model by penalizing sharp changes or oscillations in the model's gradients.

##### Gradient Accumulation
Gradient accumulation adds gradients over an effective batch of size `batch_per_iter * iters_to_accumulate` (`* num_procs` if distributed). The scale should be calibrated for the effective batch, which means inf/NaN checking, step skipping if inf/NaN grads are found, and scale updates should occur at effective-batch granularity. Also, grads should remain scaled, and the scale factor should remain constant, while grads for a given effective batch are accumulated. If grads are unscaled (or the scale factor changes) before accumulation is complete, the next backward pass will add scaled grads to unscaled grads (or grads scaled by a different factor) after which it’s impossible to recover the accumulated unscaled grads `step` must apply.

##### Automatic Mixed Precision Training
Automatic Mixed Precision (AMP) training is a technique used in deep learning to accelerate training by using a combination of lower-precision and higher-precision numerical representations. The primary idea behind AMP is to use lower-precision data types (such as float16) for some parts of the model computation, while maintaining higher precision (such as float32) for critical numerical stability aspects.

In [ ]:
def train(epoch):
    # Set the model to training mode
    model.train()
    total_loss = 0.0
    
    # Create a GradScaler once at the beginning of training.
    scaler = torch.cuda.amp.GradScaler()

    # Reset gradients
    optimizer.zero_grad()

    # Iterate over batches in the training dataloader
    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch}/Training", unit="batch") as pbar:
        for i, (images, labels) in enumerate(train_dataloader):
            # Move images and labels to the device
            images, labels = images.to(device), labels.to(device)

            # Perform forward pass with autocasting
            with torch.autocast(device_type=device.type, dtype=torch.float16, enabled=True):
                output = model(images)
                loss = criterion(output, labels)

            # # Scales the loss for autograd.grad's backward pass, producing scaled_grad_params
            # scaled_grad_params = torch.autograd.grad(outputs=scaler.scale(loss), inputs=model.parameters(), create_graph=True)
    
            # # Creates unscaled grad_params before computing the penalty. scaled_grad_params are
            # # not owned by any optimizer, so ordinary division is used instead of scaler.unscale_:
            # inv_scale = 1. / scaler.get_scale()
            # grad_params = [p * inv_scale for p in scaled_grad_params]
    
            # # Computes the penalty term and adds it to the loss
            # with torch.autocast(device_type=device.type, dtype=torch.float16, enabled=True):
            #     grad_norm = 0
            #     for grad in grad_params:
            #         grad_norm += grad.pow(2).sum()
            #     grad_norm = grad_norm.sqrt()
            #     loss = loss + grad_norm

            # Scale the loss by accumulation steps
            if ACCUMULATION_STEPS > 1:
                with torch.autocast(device_type=device.type, dtype=torch.float16, enabled=True):
                    loss = loss / ACCUMULATION_STEPS
                
            # Perform backward pass with scaling
            scaler.scale(loss).backward()

            # If the number of accumulated steps reached, update the model parameters
            if (i + 1) % ACCUMULATION_STEPS == 0:
                # Unscales gradients
                scaler.unscale_(optimizer)

                #  Applies clipping
                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
                adaptive_clip_grad(model.parameters())

                # Perform optimizer step with scaled gradients
                scaler.step(optimizer)

                # Log learning rate and update scheduler
                for i, param_group in enumerate(optimizer.param_groups):
                    mlflow.log_metric(f"lr_group{i}", param_group["lr"])
                scheduler.step()

                # Update scaler for next iteration
                scaler.update()

                # Reset gradients after optimizer step
                for param in model.parameters():
                    param.grad = None

            # Update total loss
            total_loss += loss.item()

            # Update progress bar
            pbar.update(1)
            
            # Initialize a string to store learning rates
            lr_string = ""

            # Iterate over parameter groups in optimizer
            for i, param_group in enumerate(optimizer.param_groups):
                # Append the learning rate to the string
                lr_string += f"{param_group['lr']:.5f}"
                
                # Add a comma if it's not the last parameter group
                if i < len(optimizer.param_groups) - 1:
                    lr_string += ", "

            # Set the learning rate string as the value for the 'lr' key in pbar's postfix dictionary
            pbar.set_postfix(loss=total_loss / (i+1), lr=lr_string)

    # Calculate average loss
    avg_loss = total_loss / len(train_dataloader.dataset)

    # Log average training loss
    mlflow.log_metric("train_loss", avg_loss)

    # Print epoch-wise metrics
    print(f"[Train][Epoch {epoch}] Average Loss: {avg_loss:.5f}, Updated Learning Rate: {optimizer.param_groups[0]['lr']}")

#### Eval

In [ ]:
class BestLossTracker:
    def __init__(self):
        # Initialize the best loss to positive infinity
        self.best_loss = float('inf')
    
    def update_best_loss(self, avg_loss):
        # Update the best loss if the provided loss is lower than the current best loss
        if avg_loss < self.best_loss:
            self.best_loss = avg_loss
            return True  # Return True to indicate that the best loss has been updated
        return False  # Return False if the best loss remains unchanged

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0.0):
        # Initialize parameters for early stopping
        self.patience = patience  # Number of epochs to wait before stopping
        self.delta = delta  # Minimum change in validation loss to be considered as improvement
        self.counter = 0  # Counter to keep track of epochs since last improvement
        self.best_loss = float('inf')  # Initialize the best validation loss to positive infinity
        self.early_stop = False  # Flag to indicate whether to stop early

    def update_best_loss(self, val_loss):
        # Check if the provided validation loss is better than the current best loss
        if val_loss < self.best_loss - self.delta:
            # Update the best loss
            self.best_loss = val_loss
            # Reset the counter as there's improvement
            self.counter = 0
        else:
            # Increment the counter if there's no improvement
            self.counter += 1
            # If the counter exceeds patience, set early_stop flag to True
            if self.counter >= self.patience:
                self.early_stop = True

        # Return True if training should continue (not early stopping), False otherwise
        return not self.early_stop

In [ ]:
def test(epoch, best_loss_tracker):
    # Set the model to evaluation mode
    model.eval()
    total_correct = 0
    total_loss = 0.0

    # Iterate over batches in the test dataloader
    with tqdm(total=len(test_dataloader), desc=f"Epoch {epoch}/Testing", unit="batch") as pbar: 
        with torch.no_grad():
            for i, (images, labels) in enumerate(test_dataloader):
                images, labels = images.to(device), labels.to(device)
                
                # Test-time augmentation
                augmented_predictions = []
                augmented_losses = []
                for _ in range(N_TTA):
                    augmented_images = tta_transform(images)
                    augmented_output = model(augmented_images)
                    augmented_loss = criterion(augmented_output, labels)
                    augmented_losses.append(augmented_loss.item())
                    augmented_predictions.append(augmented_output.detach().cpu().numpy())

                # Take the average loss over augmentations
                total_loss += sum(augmented_losses) / len(augmented_losses)
                
                # Take the average of predictions across augmentations
                final_predictions = torch.tensor(np.array(augmented_predictions), dtype=torch.float32).mean(dim=0)
                pred = final_predictions.detach().max(1)[1].to(device)
                total_correct += pred.eq(labels.view_as(pred)).sum()

                # Update progress bar
                pbar.update(1)

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(test_dataloader.dataset)
    accuracy = total_correct / len(test_dataloader.dataset)

    # Log validation metrics
    mlflow.log_metric("val_loss", avg_loss)
    mlflow.log_metric("val_accuracy", accuracy)

    # Update best loss tracker and save checkpoint if necessary
    if best_loss_tracker.update_best_loss(avg_loss):
        checkpoint_name = f"checkpoint_best_epoch_{epoch}.pth"
        checkpoint_path = str(SAVE_DIR / "checkpoints" / checkpoint_name)
        torch.save(model.state_dict(), checkpoint_path)
        print(f"[Test][Epoch {epoch}] New best model found! Saving checkpoint. Loss: {avg_loss:.5f}, Accuracy: {accuracy:.3f}")
    else:
        print(f"[Test][Epoch {epoch}] Loss: {avg_loss:.5f}, Accuracy: {accuracy:.3f}")
    
    return avg_loss

#### Run

In [ ]:
def run():
    # Start MLflow run
    with mlflow.start_run(run_name="Transfer Learning"):
        # Log Torch and MLflow versions
        mlflow.log_param("torch_version", torch.__version__)
        mlflow.log_param("mlflow_version", mlflow.__version__)

        # Log hyperparameters
        hyper_params = {
            "accumulation_steps": ACCUMULATION_STEPS, 
            "batch_size": BATCH_SIZE, 
            "dropout":DROPOUT_PROB,
            "epochs": EPOCHS, 
            "batchnorm": USE_BATCHNORM,
            "label_smoothing": LABEL_SMOOTHING,
            "base_lr": BASE_LR,
            "head_lr":HEAD_LR,
            "lr_decay":LR_DECAY_FACTOR,
            "warmup_proortion": WARMUP_PROPORTION,
            "weight_decay": WEIGHT_DECAY
        }
        mlflow.log_params(hyper_params)

        # Set tags
        mlflow.set_tags({"model": "lenet", "dataset": "mnist"})

        # Log the model architecture
        mlflow.pytorch.log_model(model, "model")

         # Log start time
        mlflow.log_param("start_time", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

        # Log model metrics
        model_metrics = get_model_metrics()
        mlflow.log_param("num_params", model_metrics['num_params'])
        mlflow.log_param("num_trainable_params", model_metrics['num_trainable_params'])
        mlflow.log_param("flops", model_metrics['flops'])
        mlflow.log_param("disk_size", model_metrics['disk_size'])

        # Initialize best loss tracker and early stopping
        best_loss_tracker = BestLossTracker()
        early_stopping = EarlyStopping(patience=3, delta=1e-6)

        # Start training and testing loop
        start_time = time.time()
        for epoch in range(EPOCHS):
            train(epoch)
            val_loss = test(epoch, best_loss_tracker)

            # Log GPU usage if available
            if get_gpu_usage() is not None:
                mlflow.log_metric("gpu", get_gpu_usage(), step=epoch)

            # Check for early stopping
            if not early_stopping.update_best_loss(val_loss):
                print("Early stopping triggered.")
                break

        # Calculate and log training time metrics
        total_time = time.time() - start_time
        avg_epoch_time = total_time / EPOCHS
        avg_batch_time = avg_epoch_time / len(train_dataloader)

        mlflow.log_metric("total_training_time", total_time)
        mlflow.log_metric("average_epoch_time", avg_epoch_time)
        mlflow.log_metric("average_batch_time", avg_batch_time)

        # Log end time
        mlflow.log_param("end_time", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# Dump snapshot of the current GPU memory state
torch.cuda.memory._dump_snapshot("snapshot.pickle")

In [ ]:
run()

### Evaluation

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs.to(device))
            _, predictions = torch.max(outputs, 1)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probabilities.extend(probabilities.cpu().numpy())

    return all_predictions, all_labels, all_probabilities

#### Error Analysis

In [ ]:
# Run evaluation and gather predictions, labels and probablities
classes = [str(i) for i in range(10)]
model_predictions, true_labels, model_probablities = evaluate(model, test_dataloader)

In [ ]:
def print_classification_report(y_true, y_pred, target_names):
    print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
print_classification_report(true_labels, model_predictions, classes)

### Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap=cmap, xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

In [ ]:
plot_confusion_matrix(true_labels, model_predictions, classes)

### Classwise Metrics

In [ ]:
classes = [i for i in range(10)]

In [ ]:
y_test, y_pred = np.array(true_labels), np.array(model_predictions)
for cls in classes:
    cls_indices = (y_test == cls)
    cls_precision = precision_score(y_test[cls_indices], y_pred[cls_indices], average='weighted')
    cls_recall = recall_score(y_test[cls_indices], y_pred[cls_indices], average='weighted')
    cls_f1_score = f1_score(y_test[cls_indices], y_pred[cls_indices], average='weighted')

    print(f"\nClass {cls} Metrics:")
    print(f"  Precision: {cls_precision:.4f}")
    print(f"  Recall:    {cls_recall:.4f}")
    print(f"  F1-Score:  {cls_f1_score:.4f}")

# Print overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nOverall Accuracy: {accuracy:.4f}")

### Model Interpretibility

In [ ]:
# Load the pre-trained model state dictionary from a saved checkpoint file
model.load_state_dict(torch.load("./artifacts/exp/checkpoints/checkpoint_best_epoch_8.pth"), strict=False)

# Set the model to evaluation mode
model.eval()

# The model is now loaded with weights from the specified checkpoint file and ready for inference
# Evaluation mode is set to ensure that parameters like dropout are fixed, and the model behaves deterministically during inference.

In [ ]:
# Get a batch of images and labels from the test dataloader
images, labels = next(iter(test_dataloader))
# Move the images and labels to the device (e.g., GPU if available)
images, labels = images.to(device), labels.to(device)

# Choose a specific index from the batch (e.g., the first index)
ind = 0

# Select the input image and label for the chosen index
input_image = images[ind].unsqueeze(0)
input_label = labels[ind]

# Set the 'requires_grad' attribute to True to enable gradient computation for the input
input_image.requires_grad = True

# The 'input_image' tensor is now ready for interpretation techniques that require gradient information.
# You can use this tensor as input for methods like Saliency, Integrated Gradients, etc.
# Note: 'input_label' can be used as the target label for techniques that require target information.


In [ ]:
def attribute_image_features(algorithm, input, **kwargs):
    # Ensure that gradients are cleared before attribution
    model.zero_grad()

    # Use the provided attribution algorithm to compute feature attributions
    # Here, 'input' is the input image tensor, and 'labels[ind]' is the target label
    # 'kwargs' can be used to pass additional parameters specific to the attribution algorithm
    tensor_attributions = algorithm.attribute(
        input, target=labels[ind], **kwargs
    )

    # Return the computed feature attributions
    return tensor_attributions

#### Saliency Maps

**Description:** Saliency maps highlight the input features that contribute the most to a particular output prediction. In the context of neural networks, saliency maps visualize the gradients of the output with respect to the input features.

**Use Case:** Saliency maps help identify which regions of an input image are crucial for the model's decision-making.
.

In [ ]:
saliency = Saliency(model)
grads = saliency.attribute(input_image, target=labels[ind].item())

# Visualize the saliency map
saliency_map_np = grads.squeeze(0).cpu().detach().numpy()

# Transpose the saliency map if needed
saliency_map_np = np.transpose(saliency_map_np, (1, 2, 0))

# grads = np.transpose(grads.squeeze().cpu().detach().numpy(), (1, 2, 0))

#### Integrated Gradients

**Description:** Integrated Gradients provide a way to attribute the model's output to its input features by integrating the gradients of the output with respect to the input along a path from a baseline (e.g., an input with all zeros) to the actual input.

**Use Case:** Integrated Gradients help understand how changes in input features contribute to changes in the model's outpu.


In [ ]:
ig = IntegratedGradients(model)
attr_ig, delta = attribute_image_features(ig, input_image, baselines=input_image * 0, return_convergence_delta=True)
attr_ig = np.transpose(attr_ig.squeeze(0).cpu().detach().numpy(), (1, 2, 0))
print('Approximation delta: ', abs(delta))

### Occlusion
**Description:** Occlusion involves systematically occluding or blocking parts of the input data and observing the impact on the model's output. By measuring the change in output, you can identify important regions.

**Use Case:** Helps understand which areas of the input contribute most to the model's decision, and is particularly useful for image-based models.


In [ ]:
occlusion = Occlusion(model)
occlusion_sensitivity = attribute_image_features(occlusion, input_image, sliding_window_shapes=(1, 14, 14), strides=(1, 7, 7))
occlusion_sensitivity = np.transpose(occlusion_sensitivity.squeeze(0).cpu().detach().numpy(), (1, 2, 0))

In [ ]:
# Convert the input image tensor to a NumPy array and adjust normalization
original_image = np.transpose((images[ind].cpu().detach().numpy() / 2) + 0.5, (1, 2, 0))

# Visualize the original image
_ = viz.visualize_image_attr(
    None, original_image,
    method="original_image", title="Original Image"
)

# Visualize the saliency map overlaid on the original image
_ = viz.visualize_image_attr(
    saliency_map_np, original_image,
    method="blended_heat_map", sign="absolute_value",
    show_colorbar=True, title="Overlayed Gradient Magnitudes"
)

# Visualize Integrated Gradients attributions overlaid on the original image
_ = viz.visualize_image_attr(
    attr_ig, original_image,
    method="blended_heat_map", sign="all",
    show_colorbar=True, title="Overlayed Integrated Gradients"
)

# Visualize Occlusion Sensitivity attributions overlaid on the original image
_ = viz.visualize_image_attr(
    occlusion_sensitivity, original_image,
    method="blended_heat_map", sign="all",
    show_colorbar=True, title="Occlusion Sensitivity"
)

#### Important Notes:

1. **Saliency Maps:**
   - Identify crucial regions in the input for model decisions.
   - Provides insights into feature importance.

2. **Integrated Gradients:**
   - Understands feature contributions through integration paths.
   - Helps grasp the impact of input features on model predi 3o a baseline.

5. **Occlusion:**
   - Systematically blocks input regions to observe output changes.
   - Identifies areas crucial for model decisions, especially in ###f specific features.

## Choosing the Best Technique:

- The choice depends on the interpretability goals, model complexity, and the nature of input data.
- Consider using multiple techniques for a comprehensive understanding.
- Integrated Gradients and Saliency Maps are often suitable for visualizing feature importa##nce in image-based models.

## Final Thoughts:

- Interpretability techniques provide valuable insights into neural network decision-making processes.
- No one-size-fits-all solution; the choice of technique depends on the specific use case and model characteristics.
- Experiment with different methods and evaluate their effectiveness based on your interpretability goals.

